**NOTE:** **I had to clear the outputs because github wasn't showing the code for the notebook**

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
# 1. Load Data
data = pd.read_csv('/content/drive/MyDrive/loan predictor/loan_data.csv')

In [ ]:
#Check Column Names
print(f"Columns: {data.columns.tolist()}")

Found spaces at beginning of column names

In [ ]:
# Remove spaces in coulmn names
data.columns = data.columns.str.strip()
print(f"Columns: {data.columns.tolist()}")

In [ ]:
data = data.drop(columns=['loan_id'])
data.head()

In [ ]:
# found spaces in output of loan_status column and Mapping the output of target variable
data['loan_status'] = data['loan_status'].str.strip()
data['loan_status'] = data['loan_status'].map({'Approved': 1, 'Rejected': 0})
data.head()

In [ ]:
numeric_cols = data.select_dtypes(include=['number']).columns
print("Minimum values per column:")
print(data[numeric_cols].min())

Found -ve values in res. assets column

In [ ]:
neg_mask = data['residential_assets_value'] < 0
bad_rows = data[neg_mask]

print(f"Number of negative rows: {len(bad_rows)}")
print("Unique negative values:")
print(bad_rows['residential_assets_value'].value_counts())

In [ ]:
display(bad_rows.head(10))

Looks more like a system error to me than anyother problem like debt.

I'm saying this because when you look at some data, even though the loan_status was approved or rejected. they all have same number and if we think it's debt not all of them should be having same debt amount.

In [ ]:
data = data[data['residential_assets_value'] != -100000]

In [ ]:
data['residential_assets_value'].min()

In [ ]:
print(f"Duplicates: {data.duplicated().sum()}")

In [ ]:
X = data.drop('loan_status', axis=1)
y = data['loan_status']

Splitting data into train and test sets

In [ ]:
from sklearn.model_selection import train_test_split

# Stratify=y because of the class imbalance
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

print(f"Train Shape: {X_train.shape}, Test Shape: {X_test.shape}")

In [ ]:
print("Target Class Distribution :")
y_train.value_counts(normalize=True)

Found Class Imbalance between approved and rejected

In [ ]:
X_train.describe()

In [ ]:
cat_cols = X_train.select_dtypes(include=['object']).columns.tolist()
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

In [ ]:
print(f"Categorical: {cat_cols}")
print(f"Numerical: {num_cols}")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

print("Missing Values in Numerical Columns:")
print(X_train[num_cols].isnull().sum())

print("Missing Values in Categorical Columns:")
print(X_train[cat_cols].isnull().sum())

# Nullity heatmap
plt.figure(figsize=(10, 6))
sns.heatmap(X_train.isnull(),cbar=False, cmap='viridis')
plt.title("Nullity Heatmap (Train Set)")
plt.show()


**Visualizations**

In [ ]:
# Histograms
X_train.hist(figsize=(15, 10))
plt.show()

Outlier check using box plots

In [ ]:
# Boxplots (Outliers)
for col in num_cols:
    plt.figure()
    sns.boxplot(x=X_train[col])
    plt.title(f"Boxplot of {col}")
    plt.show()

I found outliers in these 3 columns bank_asset_value, residential_assets_value, and commercial_assets_value which are real outliers.


people with high income will have more valuable assets. they are rich maybe millionaire or billionaire.

In [ ]:
#Correlation Heatmap codedat
plt.figure(figsize=(12, 8))
corr_matrix = X_train[num_cols].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap')
plt.show()

Found strong correlation bet

In [ ]:
def create_features(df):
    df = df.copy()
    #Loan to Income ratio
    df['loan_to_income_ratio'] = df['loan_amount'] / df['income_annum']

    #Aggregate Assets
    df['total_assets'] = (df['residential_assets_value'] +
                          df['commercial_assets_value'] +
                          df['luxury_assets_value'] +
                          df['bank_asset_value'])

    # 3. Drop Redundant Columns
    df = df.drop(columns=['loan_amount', 'income_annum',
                          'residential_assets_value', 'commercial_assets_value',
                          'luxury_assets_value', 'bank_asset_value'])
    return df

# Apply to Train AND Test
X_train = create_features(X_train)
X_test = create_features(X_test)

#Updating columns after feature engineering
cat_cols = X_train.select_dtypes(include=['object']).columns.tolist()
num_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()

# check Correlation on the NEW features
plt.figure(figsize=(12, 8))
corr_matrix = X_train[num_cols].corr()
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', fmt=".2f")
plt.title('Correlation Heatmap (Post-Feature Engineering)')
plt.show()

I performed feature engineering to address the multicollinearity I found in the heatmap.


1. Interaction Feature: I created a loan_to_income ratio. This captures the applicant's repayment burden, which is a much stronger predictor of risk than raw income alone.


2. Dimensionality Reduction: Since the asset classes were highly correlated, I aggregated them into total_assets.

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', num_cols),
        ('cat', OneHotEncoder(handle_unknown='ignore'), cat_cols)
    ])

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report,accuracy_score

models = {
    "Decision Tree": DecisionTreeClassifier(random_state=42, class_weight='balanced'),
    "Random Forest": RandomForestClassifier(random_state=42, class_weight='balanced'),
    "XGBoost": XGBClassifier(eval_metric='logloss', scale_pos_weight=10)
    # Note: XGBoost uses 'scale_pos_weight' instead of 'class_weight' for imbalance
}

In [ ]:
for name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])

    # Fit the pipeline
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    print(f"--- {name} Report ---")
    print(classification_report(y_test, y_pred))
    print("\n")

In [ ]:
# overfitting Check
for name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('classifier', model)
    ])

    pipeline.fit(X_train, y_train)

    # Train score
    y_train_pred = pipeline.predict(X_train)
    train_score = accuracy_score(y_train, y_train_pred)

    # Test score
    y_test_pred = pipeline.predict(X_test)
    test_score = accuracy_score(y_test, y_test_pred)

    print(f"\n--- {name} ---")
    print(f"Train Accuracy: {train_score:.4f}")
    print(f"Test Accuracy: {test_score:.4f}")
    print(f"Difference: {abs(train_score - test_score):.4f}")
    print(classification_report(y_test, y_test_pred))

In [ ]:
#XGBoost pipeline
model = Pipeline(steps=[('preprocessor', preprocessor), ('classifier', models['XGBoost'])])
model.fit(X_train, y_train)

import shap

# 1. Setup Explainer
explainer = shap.TreeExplainer(model.named_steps['classifier'])

# 2. Transform Data
X_train_transformed = model.named_steps['preprocessor'].transform(X_train)
if hasattr(X_train_transformed, 'toarray'):
    X_train_transformed = X_train_transformed.toarray()

# 3. Calculate SHAP values
shap_values = explainer.shap_values(X_train_transformed)

# 4. Summary Plot
shap.summary_plot(shap_values, X_train_transformed, feature_names=model.named_steps['preprocessor'].get_feature_names_out())

# 5. Force Plot
shap.initjs()
shap.force_plot(explainer.expected_value, shap_values[0,:], X_train_transformed[0,:], feature_names=model.named_steps['preprocessor'].get_feature_names_out())

In [ ]:
# model saving
import joblib

model_path = '/content/drive/MyDrive/loan predictor/loan_model.joblib'
joblib.dump(model, model_path)

In [ ]:
from huggingface_hub import HfApi, login

HF_TOKEN = "TOKEN"
login(HF_TOKEN)

repo_name = "Ravichandrachilde/loan-prediction-XGB"

api = HfApi()
api.upload_file(
    path_or_fileobj="/content/drive/MyDrive/loan predictor/loan_model.joblib",
    path_in_repo="loan_model.joblib",
    repo_id=repo_name,
    token=HF_TOKEN,
    repo_type="model"
)